SARIMA MODEL:

In [8]:
# import yahoo finance data
import yfinance as yf
# import stockstats data
from stockstats import StockDataFrame as ss

# import necessary libraries
import matplotlib as mp
import numpy as np
import pandas as pd
import pytz
import warnings
import time
import random
import statistics
import pydoc
import os
import pyarrow
import pandas_gbq
import statsmodels
import tensorflow

#import libraries for SARIMA model
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle as pkl
from datetime import datetime
from datetime import timedelta
from datetime import date

In [9]:
apple_ticker = yf.Ticker("AAPL")
apple_data = yf.download("AAPL", start = '2020-01-01', interval = '1d')
apple_df = ss.retype(apple_data)

apple_data[['stochrsi', 'macd', 'mfi']] = apple_df[['stochrsi', 'macd', 'mfi']]
print(apple_data)
print(apple_ticker.get_capital_gains)

[*********************100%%**********************]  1 of 1 completed
                  open        high         low       close   adj close  \
Date                                                                     
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.152657   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.441444   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.018700   
2020-01-07   74.959999   75.224998   74.370003   74.597504   72.675278   
2020-01-08   74.290001   76.110001   74.290001   75.797501   73.844345   
...                ...         ...         ...         ...         ...   
2024-01-24  195.419998  196.380005  194.339996  194.500000  194.500000   
2024-01-25  195.220001  196.270004  193.110001  194.169998  194.169998   
2024-01-26  194.270004  194.759995  191.940002  192.419998  192.419998   
2024-01-29  192.009995  192.199997  189.580002  191.729996  191.729996   
2024-01-30  190.940002  191.800003  187.470

In [10]:
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

# import google cloud service account and bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

# specify google cloud project information
credentials = service_account.Credentials.from_service_account_file(
    'black-vehicle-406619-bf2e31773163.json')
project_id = 'black-vehicle-406619'
client = bigquery.Client(project=project_id, credentials=credentials)
dataset_id = 'stocks_ds'
table_id = '20yrs_stockdata'
table_path = f"{project_id}.{dataset_id}.{table_id}"

# specify load reqs
load_info = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
load_data = client.load_table_from_dataframe(apple_data, table_path, job_config=load_info)
load_data.result()

LoadJob<project=black-vehicle-406619, location=US, id=3ec432d6-2982-4390-b1e8-5704b301ee89>

In [5]:
#Seasonal - fit stepwise auto-ARIMA
#!pip install pmdarima

# Remove any duplicate index
apple_data = apple_data.loc[~apple_data.index.duplicated(keep='first')]

#Filter only required data

apple_data = apple_data[['close']]

In [6]:
#Scale the APPL data into a standard range using MinMaxScaler ()

Feature_Scaler = MinMaxScaler()

#Transform current APPL data

apple_transformed = pd.DataFrame(np.squeeze(Feature_Scaler.fit_transform(apple_data), axis=1), columns=["Close"], index=apple_data.index)


In [7]:

sarima_model = pm.auto_arima(apple_transformed["Close"], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3,
                         m=12, #12 is the frequency of the cycle (yearly)
                         start_P=0,
                         seasonal=True, #set to seasonal
                         d=None,
                         D=1, #order of the seasonal differencing
                         trace=False,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)

# start_p=1, start_q=1: Sets the initial values for the order of the AR (AutoRegressive) and MA (Moving Average) components in the non-seasonal part of the model.
# test='adf': Specifies the use of the Augmented Dickey-Fuller (ADF) test to determine whether the time series is stationary and to help in determining the need for differencing (`d` parameter).
# max_p=3, max_q=3: Specifies the maximum values for the `p` and `q` parameters to consider during the model fitting process.
# start_P=0: Sets the initial value for the order of the seasonal AR component.
# d=None: The order of non-seasonal differencing is not specified, which allows the function to determine it automatically.
# trace=False: This means that the function will not print out diagnostic information about the steps it's taking.
# error_action='ignore': Instructs the function to ignore errors and try different combinations of parameters.
# suppress_warnings=True: Suppresses convergence warnings, which can be frequent in ARIMA modeling.
# stepwise=True: Enables a stepwise search to efficiently find the best model parameters.

In [7]:
sarima_model.plot_diagnostics(figsize=(15,12))
plt.show()

NameError: name 'sarima_model' is not defined

In [5]:
import pickle
# Serialize with Pickle and save it as pkl
with open('sarima_model.pkl', 'wb') as pkl:
    pickle.dump(sarima_model, pkl)

# Desiarilize the content of the file back into a Python object
with open('sarima_model.pkl', 'rb') as pkl:
    loaded_model = pickle.load(pkl)

NameError: name 'sarima_model' is not defined

In [10]:
def forecast(model, df, forecast_date):
    # Forecast
    n_periods = (datetime.strptime(forecast_date, '%Y-%m-%d') - df.index[-1]).days
    fitted, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(df.index[-1] + pd.DateOffset(days=1), periods = n_periods, freq='D')

    # Make series for plotting purpose
    fitted_series = pd.Series(fitted.values, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    #Concatenate the original DataFrame with forecasted values and confidence intervals
    df_result = pd.concat([df, fitted_series, lower_series, upper_series], axis=1)
    df_result.columns = ["Actual", "Prediction", "Low", "High"]

    #Inverse transform the scaled values to their original scale
    for column in df_result.columns:
      df_result[column] = Feature_Scaler.inverse_transform(df_result[column].values.reshape(-1,1))


    return df_result

In [15]:
test=forecast(loaded_model, apple_transformed, '2024-03-01')

/home/wjulia800/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/wjulia800/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
test

In [ ]:
# Plot
plt.figure(figsize=(15,7))
plt.plot(test["Actual"][-120:], color='#1f76b4')
plt.plot(test["Prediction"], color='darkgreen')
plt.fill_between(test.index,
                test["Low"],
                test["High"],
                color='k', alpha=.15)

plt.title("SARIMA - Forecast of APPL Stock Price")
plt.show()

LSTM MODEL:

In [ ]:
apple_data = yf.download("AAPL", start = "2020-01-01", interval = '1d')
apple_df_test = apple_data.reset_index()
apple_df = apple_df_test.copy()

In [ ]:
scaler = MinMaxScaler()
data_transformed = pd.DataFrame(
    np.squeeze(
        scaler.fit_transform(
            apple_df[["Close"]])), columns=["Close"], index=apple_df.index)

In [ ]:
def features_targets(data, feature_length):
    # feature length is the number of time steps in the input sequence
    # targets are the values the model is trying to forecast
    time_step_list, close_label_list = [], []
    
    # iterate through (length of sequential data) to (length of seq data - feature length)
    for i in range(len(data) - feature_length):
        # this will get the vals leading up to the target
        time_steps = data[i : i + feature_length]
        time_step_list.append(time_steps)
        # this will get the target val at this point
        labels = data[i + feature_length]
        close_label_list.append(labels)

    # reshape lists to be suitable for network algo
    time_step_list = np.array(time_step_list).reshape(len(time_step_list), feature_length, 1)
    close_label_list = np.array(close_label_list).reshape(len(close_label_list), 1)

    return time_step_list, close_label_list

In [ ]:
time_step_vals, target_vals = features_targets(data_transformed["Close"].values, 20)

In [ ]:
train_vals_cutoff = apple_df.loc[apple_df['Date'] >= '2022-01-01']
slice = train_vals_cutoff.shape[0]


In [ ]:
def create_model(X, Y, df, data, train_test_slice, scaler):
    # training set: set to train the machine learning model
    # testing set: set used to test model after model has been trained
    # train set is 70% of data, test set is the rest (30%)
    X_train, X_test = X[:-train_test_slice], X[-train_test_slice:]
    Y_train, Y_test = Y[:-train_test_slice], Y[-train_test_slice:]

    # initialize empty model where nodes have input and output with Keras
    model = Sequential()
    # create a bidirectional LSTM: 
    # - 100 cells
    # - return output for input
    # - reduce overfitting w current dropout
    # - specify # of steps for target
    model.add(Bidirectional(LSTM(100, return_sequences=True, recurrent_dropout=0.1, input_shape=(X_train.shape[1], 1))))
    # provide additional processing with undirectional layer
    model.add(LSTM(50, recurrent_dropout=0.1))
    # add dropout and dense layers
    # randomly sets 20% of inputs to 0 to prevent overfitting
    model.add(Dropout(0.2))
    # create a connected layer with 25 output units
    model.add(Dense(20, activation='elu'))
    model.add(Dropout(0.2))
    # create a connected layer with 10 output units
    model.add(Dense(10))
    # create a connected layer with 1 output unit
    model.add(Dense(1))

    # optimize model using stochastic gradient descent to train model
    # SGD 
    optimize = tf.keras.optimizers.SGD(learning_rate = 0.002)
    # compile model
    model.compile(loss='mean_squared_error', optimizer=optimize)
    # save model weights validation loss improves
    weights = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
    # adjust learning rate when needed
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4, min_lr=0.00001, verbose=1)

    # one epoch completes when the entire training dataset is processed once by the model
    model.fit(X_train, Y_train, epochs=12, batch_size=1, verbose=1, shuffle=False, validation_data=(X_test, Y_test), callbacks=[reduce_lr, weights])
    actual = scaler.inverse_transform(Y_test)
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    actual = np.squeeze(actual, axis=1)
    predictions = np.squeeze(predictions, axis=1)

    #reassign index before it goes into model
    test_df = pd.DataFrame({'Actual': actual, 'Predicted': predictions.flatten()})
    
    #print test_df plot
    '''
    # Plotting test set
    graph.plot(df.index[-train_test_slice:], predictions, label="Predicted")
    graph.plot(df.index[-train_test_slice:], actual, label="Actual")
    graph.xlabel('Date')
    graph.ylabel('Stock Price')
    graph.legend()
    graph.savefig('predicted_stock_prices_lstm3_test.png')
    graph.show()
    '''
    return model, X_train, X_test, Y_train, Y_test

In [ ]:
# assign model, x training set, y training set, x testing set, and y testing set
lstm_model, X_train, X_test, Y_train, Y_test = create_model(
        time_step_vals, target_vals, apple_df, data_transformed["Close"].values, slice, scaler)

In [ ]:
# concatenate x sets, horiz axis
total_x = np.concatenate((X_train, X_test), axis = 0)
# concatenate y sets, horiz axis
total_y = np.concatenate((Y_train, Y_test), axis = 0)
# predict x axis with predict function
final_predict = model.predict(total_x)
# inverse transform x axis predictions
final_predict = scaler.inverse_transform(final_predict)
# inverse transform y predictions
actual = scaler.inverse_transform(total_y)
final_predict = np.squeeze(final_predict, axis = 1)
actual = np.squeeze(actual, axis = 1)

In [ ]:
graph.plot(final_predict, label = "total predict")
graph.plot(actual, label = "total actual")
graph.xlabel('Date')
graph.ylabel('Stock Price')
graph.legend()
graph.savefig('test.png')
graph.show()

In [ ]:
def predict_lstm(model, df, future_date, scaler, feature_length=20):
    # iterate through today's date until future date
    for i in range((datetime.strptime(future_date, '%Y-%m-%d') - df.index[-1]).days):
        # specify close values
        feature_column = df['Close'].values
        # pick out last 20 days
        time_steps = feature_column[-feature_length:]
        # reshape array
        time_steps = time_steps.reshape(feature_length, 1)
        # scale array
        time_steps = scaler.transform(time_steps)
        prediction = model.predict(time_steps.reshape(1, feature_length, 1))
        prediction = scaler.inverse_transform(prediction)
        # concatenate results with og dataframe
        predicted_vals = pd.DataFrame(prediction, index=[df.index[-1] + timedelta(days=1)], columns=['Close'])
        df = pd.concat([df, predicted_vals])
    return df

In [ ]:
# reshape model's last twenty values into array of 1 row, 20 cols, 1 scalar (cause of chosen feature length)
data_transformed = data_transformed[-21:-1].values.reshape(1,20,1)
# predict model with predict function
lstm_model.predict(data_transformed)
# create new df out of apple_data
#data = pd.DataFrame(apple_data)
test = predict_lstm(lstm_model, apple_df, '2024-02-01', scaler)

In [ ]:
test.tail()

In [ ]:
def combined_model(data_transformed, lstm_model, sarima_model, forecast_date, scaler):

  # create dataframes for high and low values
  df_high = pd.DataFrame(columns=['High'])
  df_low = pd.DataFrame(columns=['Low'])
  #last_initial_date = data_transformed.index[-1]
  # create variable referring to last day
  last_day = data_transformed.index[-1]

  # iterate through the number of days between last day and future date
  for i in range(datetime.strptime(forecast_date, '%Y-%m-%d') - last_day.days):
    # call prediction function for lstm
    df_lstm_temp = predict_lstm(lstm_model, data_transformed, (last_day + timedelta(days=1)).strftime('%Y-%m-%d'), scaler)
    # call forecast function for sarima
    df_sarima_temp = forecast(sarima_model, data_transformed, (last_day + timedelta(days=i+1)).strftime('%Y-%m-%d'), scaler)

    # combine 40% of lstm results and 60% of sarima results for closing vals on last day (day to predict)
    combination = 0.4 * (scaler.transform(df_lstm_temp.iloc[-1]['Close'].reshape(1,-1))) + 0.6 * (scaler.transform(df_sarima_temp.iloc[-1]['Prediction'].reshape(1,-1)))

    # create new dataframe of combo :D
    forecast = pd.DataFrame(combination, index = [data_transformed.index[-1] + timedelta(days=1)], columns = ['Close'])
    data_transformed = pd.concat([data_transformed, forecast])
    # create new dataframe of low values
    df_low = pd.concat([df_low, pd.DataFrame(df_sarima_temp.iloc[-1]['Low'],
                                          index=[data_transformed.index[-1]+ timedelta(days=1)],
                                          columns=['Low'])
    ])
    # create new dataframe of high values
    df_high = pd.concat([df_high, pd.DataFrame(df_sarima_temp.iloc[-1]['High'],
                                          index=[data_transformed.index[-1]+ timedelta(days=1)],
                                          columns=['High'])
    ])

  # squeeze final results to one axis, and inverse transform
  df_final = pd.DataFrame(np.squeeze(scaler.inverse_transform(data_transformed)),
                        index=data_transformed.index, columns=['Close'])

  # final actual df values are all days leading up to last day that has occurred
  df_final_actual = df_final[:last_day + timedelta(days=1)]
  # final predicted df values are all future days
  df_final_prediction = df_final[last_day + timedelta(days=1):]

  return df_final_actual, df_final_prediction, df_low,  df_high

In [ ]:
# get ~final~ combined results
combined_actual, combined_prediction, combined_df_low, combined_df_high = combined_model(data_transformed, lstm_model, sarima_model, '2024-03-01', scaler)

In [ ]:

# specify google cloud project information
dataset_id = 'predicted_prices'
table_id = 'SARIMA and LTSM Predicted Prices'
table_path = f"{project_id}.{dataset_id}.{table_id}"

# specify load reqs
load_info = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
# enter combined data load_data = client.load_table_from_dataframe(combined data, table_path, job_config=load_info)
load_data.result()